# Social Network info preprocess

In [1]:
import numpy as np
import pandas as pd
import csv

data_ctt = pd.read_csv(r'real_tweets.csv',nrows =6963,encoding='UTF-8')
with open(r'real_tweets.csv',encoding='UTF-8') as csvfile:
    reader = csv.reader(csvfile)
    rows=[]
    for i,row in enumerate(reader):
        if i>6963:
            rows.append(row)
label = ['_id','user_id','like_num','repost_num','comment_num','video_url','origin_weibo','content']
data_left = pd.DataFrame(rows[1:],columns=label)
data_left = data_left.drop(['video_url','origin_weibo'], axis = 1)
frames=[data_ctt,data_left]
dataset = pd.concat(frames)

In [2]:
usr_info = pd.read_csv(r'real_info.csv',encoding='UTF-8')
usr_info['is_spammer'] = 'no'

fake_info = pd.read_csv(r'fake_info.csv',encoding='UTF-8')
fake_info['is_spammer']='yes'


frames=[usr_info,fake_info]
usr_info = pd.concat(frames,ignore_index=True)


dataset_fake = pd.read_csv(r'fake_tweets.csv',encoding='UTF-8')
dataset_fake = dataset_fake.drop(['origin_weibo'], axis = 1)
frames=[dataset,dataset_fake]
dataset = pd.concat(frames,ignore_index=True)
usr_info['follow_ratio'] = usr_info.follows_num / usr_info.fans_num


In [4]:
dataset.to_csv('tweets_total.csv')

In [131]:
usr_info['class']=0#会员等级
for i,row in enumerate(usr_info._id):
    if usr_info.vip_level[i][0] in '123456789':
        usr_info.loc[i,'class']=int(usr_info.vip_level[i][0])

In [132]:
# 被转评赞的总数，平均数
usr_info['like_count']=0
usr_info['repost_count']=0
usr_info['comment_count']=0
usr_info['tweet_count']=0
usr_info['ave_like']=0.
usr_info['ave_repost']=0.
usr_info['ave_comment']=0.

usr_info['att_count']=0
usr_info['jing_count']=0
usr_info['zhuanfa_count']=0
usr_info['url_count']=0
usr_info['ave_att']=0.
usr_info['ave_jing']=0.
usr_info['ave_zhuanfa']=0.
usr_info['ave_url']=0.


for i,usr_id in enumerate(usr_info._id):
    # usr_tweet = []
    tweet_num=0
    usr_like=0
    usr_repost=0
    usr_comment=0
    for j,tweet_id in enumerate(dataset.user_id):
        if usr_id == tweet_id:
            # print(dataset.content[j:j+1].to_string())
            if '#' in dataset.content[j:j+1].to_string():
                usr_info.loc[i,'jing_count'] +=1
            if '@' in dataset.content[j:j+1].to_string():
                usr_info.loc[i,'att_count'] +=1
            if 'http' in dataset.content[j:j+1].to_string():
                usr_info.loc[i,'url_count'] +=1
            if '转发微博' in dataset.content[j:j+1].to_string():
                usr_info.loc[i,'zhuanfa_count'] +=1
            # usr_tweet.append(dataset.content[j:j+1]) #找该用户发的所有的推文
            tweet_num+=1
            # print(dataset.like_num[i:i+1])
            usr_like+=int(dataset.like_num[j:j+1].tolist()[0])
            usr_comment+=int(dataset.comment_num[j:j+1].tolist()[0])
            usr_repost+=int(dataset.repost_num[j:j+1].tolist()[0])
    # abort()
    usr_info.loc[i,'like_count'] = usr_like
    usr_info.loc[i,'repost_count'] = usr_repost
    usr_info.loc[i,'comment_count'] = usr_comment
    usr_info.loc[i,'tweet_count'] = tweet_num
    if tweet_num>0:
        usr_info.loc[i,'ave_like'] = usr_like/tweet_num
        usr_info.loc[i,'ave_comment'] = usr_comment/tweet_num
        usr_info.loc[i,'ave_repost'] = usr_repost/tweet_num
        usr_info.loc[i,'ave_jing'] = usr_info.loc[i,'jing_count']/tweet_num
        usr_info.loc[i,'ave_att'] = usr_info.loc[i,'att_count']/tweet_num
        usr_info.loc[i,'ave_url'] = usr_info.loc[i,'url_count']/tweet_num
        usr_info.loc[i,'ave_zhuanfa'] = usr_info.loc[i,'zhuanfa_count']/tweet_num


In [134]:
# usr_info
print(sum(usr_info.tweet_count>0))


372


In [135]:
usr_info.to_csv('test.csv')

In [100]:

fake_info['is_spammer'] = 'yes'
fake_info['follow_ratio'] = fake_info.follows_num / fake_info.fans_num

In [102]:
fake_info['class']=0#会员等级
for i,row in enumerate(fake_info._id):
    # print(fake_info.vip_level[i],i)
    if fake_info.vip_level[i][0] in '123456789':
        fake_info.loc[i,'class']=int(fake_info.vip_level[i][0])

In [43]:
usr_info = pd.read_csv(r'usr_info.csv',encoding='UTF-8')
# usr_info = usr_info.drop(["_id",'nick_name','vip_level'], axis = 1) #把user id等无关变量去掉

real_rela = pd.read_csv(r'real_relationships.csv')
fake_rela = pd.read_csv(r'fake_relationships.csv')

frames = [real_rela, fake_rela]
rela = pd.concat(frames,ignore_index=False)

rela

,_id,fan_id,followed_id
0,6401018685-5936379789,6401018685,5936379789
1,5997605837-5936379789,5997605837,5936379789
2,7039931990-5936379789,7039931990,5936379789
3,5982896175-5936379789,5982896175,5936379789
4,7189118589-5936379789,7189118589,5936379789
...,...,...,...
14184,5477004387-2404998983,5477004387,2404998983
14185,3204296454-2404998983,3204296454,2404998983
14186,6970481730-2404998983,6970481730,2404998983
14187,6457462173-2404998983,6457462173,2404998983


是否关注新手指南（这个号的粉丝太多了，污染了我们的数据）

粉丝的粉丝平均数

关注者的粉丝平均数

In [44]:
# usr_info
usr_info['fan_fan_ave']=0#粉丝的粉丝平均数
usr_info['followed_fan_ave']=0#关注者的粉丝平均数
usr_info['mutual']=0#互关数
usr_info['sina_helper_fan']=False
for i ,row in enumerate(usr_info._id):
    fan_data = rela.loc[rela['followed_id'] == row]
    follower_data = rela.loc[rela['fan_id'] == row]
    fans_num=0
    fans_tot = 0
    follows_num=0
    follows_tot = 0
    mutual = 0
    if not fan_data.empty:
        for j, fanner_id in enumerate(fan_data.fan_id):
            fan_info = usr_info.loc[usr_info._id == fanner_id]
            if not fan_info.empty and fan_info.fans_num.values[0]<1e8:
                fans_num += fan_info.fans_num.values[0]
                fans_tot+=1
            if not fan_info.empty and fan_info.fans_num.values[0]>1e8:
                usr_info.loc[i,'sina_helper_fan']=True
        if fans_tot!=0:
            usr_info.loc[i,'followed_fan_ave'] = fans_num / fans_tot

    if not follower_data.empty:
        for j, follower_id in enumerate(follower_data.followed_id):
            follower_info = usr_info.loc[usr_info._id == follower_id]
            if not follower_info.empty:
                follows_num += follower_info.fans_num.values[0]
                follows_tot+=1
        if follows_tot!=0:
            usr_info.loc[i,'fan_fan_ave'] = follows_num / follows_tot
    if not fan_data.empty and not follower_data.empty:
        for j, fanner_id in enumerate(fan_data.fan_id):
            fan_info = follower_data.loc[follower_data.followed_id == fanner_id]#是否互关
            if not fan_info.empty:
                mutual +=1
        usr_info.loc[i,'mutual'] = mutual


In [46]:
usr_info.loc[usr_info.followed_fan_ave>1]

,Unnamed: 0,_id,nick_name,vip_level,tweets_num,follows_num,fans_num,is_spammer,follow_ratio,class,...,zhuanfa_count,url_count,ave_att,ave_jing,ave_zhuanfa,ave_url,fan_fan_ave,followed_fan_ave,mutual,sina_helper_fan
4,4,5591276041,BardScarborough,1级,160,402,855,no,0.470175,1,...,1,0,0.000000,0.111111,0.027778,0.000000,157.000000,157.000000,1,False
6,6,3286784235,李雪晔_,3级,218,850,1094,no,0.776965,3,...,0,1,0.011494,0.195402,0.000000,0.005747,521.500000,322.500000,1,False
7,7,2036482975,橙子的紫色棋子-,5级,108,299,5924,no,0.050473,5,...,5,0,0.009346,0.028037,0.046729,0.000000,0.000000,163.500000,0,False
9,9,3780111293,Ah青青青青青_,未开通,27,264,348,no,0.758621,0,...,0,0,0.000000,0.000000,0.000000,0.000000,157.000000,157.000000,1,False
12,12,5021616718,G_萬事勝意,4级,317,838,316,no,2.651899,4,...,77,0,0.315961,0.133550,0.250814,0.000000,96.666667,131.250000,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,570,3274087355,吃喝玩乐的创客,未开通,37,351,143,yes,2.454545,0,...,0,0,0.000000,0.000000,0.000000,0.000000,436.000000,436.000000,1,False
568,575,5407064237,峰之声2014,未开通,614,884,182,yes,4.857143,0,...,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,3182.000000,0,False
569,576,6683924140,梁仁进20999,未开通,60,3847,348,yes,11.054598,0,...,0,0,0.000000,0.000000,0.000000,0.000000,3182.000000,3182.000000,28,False
570,577,5528649204,-Hu___,未开通,145,4595,436,yes,10.538991,0,...,0,0,0.000000,0.000000,0.000000,0.000000,3119.222222,22321.666667,9,False


In [53]:
usr_info = usr_info.drop(['Unnamed: 0'],axis=1)
usr_info.to_csv('new_usr_info.csv',index=False)
rela.to_csv('relationships.csv',index=False)